# Initialize

In [ ]:
# %matplotlib 
%load_ext autoreload
%autoreload 2

In [ ]:
"""
Investigate the behavior of connected particles through statistical tests based on a periodogram
"""

# try:
#     bl_has_run
#
# except Exception:
#     %matplotlib
#     %load_ext autoreload
#     %autoreload 2
#     bl_has_run = True

import cProfile
import os
import pickle
import pstats
import re

import matplotlib.pyplot as plt
import numpy as np
from numpy import arctan, cos, log, pi, sin
from scipy.fftpack import fft
from scipy.optimize import minimize, root_scalar

# eqn(sol.root, sol.root / (alpha + 1)) / eqn(sol.root, D_interval[1])
from scipy.special import gammaln
from tqdm import tqdm
import logging

from likelihood import (
    estimate_sigma2_matrix,
    get_ln_likelihood_func_2_particles_x_link,
    get_ln_likelihood_func_no_link,
    get_MLE,
    get_sigma2_matrix_func,
)

#  Load and plot data of the test result as a function of the link strength
from plot import (
    plot_angle_dependence,
    plot_diffusivity_dependence,
    plot_link_strength_dependence,
    plot_localization_dependence,
    plot_periodogram,
    contour_plot_free_dumbbell_same_D,
    contour_plot_localized_eta12_v_eta1_eta2,
    contour_plot_localized_eta12_v_gamma,
    contour_plot_localized_eta12_v_eta_ratio,
    contour_plot_localized_gamma_v_eta,
    contour_plot_localized_gamma_v_eta_ratio,
    contour_plot_localized_eta12_v_M,
    contour_plot_localized_gamma_v_M,
    contour_plot_localized_eta12_v_eta,
)
from simulate import simulate_2_confined_particles_with_fixed_angle_bond
from support import (
    calculate_min_number_of_tries_with_a_binomial_model,
    get_rotation_matrix,
    locally_rotate_a_vector,
)
from EnergyTransferResults import EnergyTransferResults

arguments_file = "arguments.dat"


def clear_arguments_file(cluster):
    if cluster:
        try:
            os.unlink(arguments_file)
        except OSError:
            pass


#  Constants
D2 = 0.4  # um^2/s
D1 = 5 * D2  # um^2/s; 0.4
n1 = 2e3
n2 = 2e3
n12 = 10 * n2  # s^{-1}. Somehting interesting happens between [1e-9; 1e-6]
N = 101  # required number of points in a trajectory
dt = 0.05  # s 0.3
gamma = 1e-8  # viscous drag, in kg/s
L = 0.5
angle = 0
trial = 0  # the trial number


k1, k2, k12 = np.array([n1, n2, n12]) * gamma
T = dt * (N - 1)  # s
M = N - 1

true_parameters = {
    name: val
    for name, val in zip(
        ("D1 D2 n1 n2 n12 gamma T dt angle L trial M".split()),
        (D1, D2, n1, n2, n12, gamma, T, dt, angle, L, trial, M),
    )
}

In [ ]:
recalculate_trajectory = 0
cluster = 1
group_calculation = True

clear_arguments_file(cluster)
trials_low_res = 100
trials_high_res = 200
resolution = 4
Ms = (1000,)
gamma_range = (1e-2, 100)
eta12_range = (1e-3, 100)
eta_range = (1e-2, 100)
# eta_ratio_range= (1e-3, 1e3)
M_range = list(range(1, 10)) + list(range(10, 100, 10)) + list(range(100, 1000, 100))

l0_range = [3e-1, 1e4]


if recalculate_trajectory:
    logging.warning("Trajectories will be recalculated!")

## Testing the new free Hookean dumbbell model

In [ ]:
clear_arguments_file(cluster)
lgbfvals = contour_plot_free_dumbbell_same_D(
    trials=100,
    Ms=[1000],
    l0_range=l0_range,
    eta_range=eta_range,
    #                                              verbose=False, recalculate_trajectory=recalculate_trajectory,
    cluster=cluster,
    dt=0.5,
)

In [ ]:
# # recalculate_trajectory = 0
# # # recalculate_BF = 1
# # cluster = 1

# # l0_range=[1e-1, 1e5]
# # eta_range=[1e-2, 200]

# clear_arguments_file(cluster)

# #
# lgbfvals = contour_plot_free_dumbbell_same_D(trials=10, Ms=[200, 1000], l0_range=l0_range, eta_range=eta_range,
#                                              verbose=False, recalculate_trajectory=recalculate_trajectory,
#                                       cluster=cluster, dt = 0.05)

# # l0_range=[1e-1, 1e4], eta_range=[1e-5, 1e3]
# # l0_range=[1e-1, 1e3], eta_range=[1e-2, 1e2]

# Test for energy transfer

In [ ]:
# WARNING: removes all loaded results
energy_transfer_results = {}

In [ ]:
from plot import contour_plot_localized_eta12_v_gamma_energy_transfer_triple_test

clear_arguments_file(cluster)

energy_transfer_figures_list = []

## Link strength and diffusivity ratio $\lg B(\eta_{12}, \gamma \equiv D_1/D_2)$

Class version of energy transfer scheduling

In [ ]:
name = "localized_gamma_v_eta12_energy_transfer"


def update_x(args_dict, z):
    args_dict.update({"n12": z / dt})


def update_y(args_dict, z):
    D1 = args_dict["D2"] * z
    args_dict.update({"D1": D1})


if not group_calculation:
    clear_arguments_file(cluster)

if name not in energy_transfer_results.keys():
    energy_transfer_results[name] = EnergyTransferResults(
        trials=trials_high_res,
        resolution=resolution,
        verbose=False,
        cluster=cluster,
        Ms=Ms,
        x_label=r"$\eta_{12}$",
        x_range=eta12_range,
        update_x=update_x,
        y_label=r"$\gamma \equiv D_1/D_2$",
        y_range=gamma_range,
        update_y=update_y,
        dt=0.05,
        angle=0,
        rotation=True,
        print_MLE=False,
        title="n1={n1:.2f}, n2={n2:.2f}, D2={D2:.2f},\ndt={dt}, L0={L0:.2f}",
        figname_base=name,
    )
    energy_transfer_figures_list.append(name + ".pdf")

energy_transfer_results[name].run()

## Link strength and localization $\lg B(\eta_{12}, \eta_1)$

In [ ]:
name = "localized_eta12_v_eta1_energy_transfer"


def update_y(args_dict, z):
    args_dict.update({"n12": z / dt})


def update_x(args_dict, z):
    args_dict.update({"n1": z / dt})


if not group_calculation:
    clear_arguments_file(cluster)

if name not in energy_transfer_results.keys():
    energy_transfer_results[name] = EnergyTransferResults(
        trials=trials_high_res,
        resolution=resolution,
        verbose=False,
        cluster=cluster,
        Ms=Ms,
        y_label=r"$\eta_{12}$",
        y_range=eta12_range,
        update_y=update_y,
        x_label=r"$\eta_1$",
        x_range=eta_range,
        update_x=update_x,
        dt=0.05,
        angle=0,
        rotation=True,
        print_MLE=False,
        title="D1={D1:.2f},\nD2={D2:.2f}, dt={dt}, L0={L0:.2f}, eta1={eta1:.2f}",
        figname_base=name,
    )
    energy_transfer_figures_list.append(name + ".pdf")

energy_transfer_results[name].run()

## Link strength and localization $\lg B(\eta_{12}, \eta_2)$

In [ ]:
name = "localized_eta12_v_eta2_energy_transfer"


def update_y(args_dict, z):
    args_dict.update({"n12": z / dt})


def update_x(args_dict, z):
    args_dict.update({"n2": z / dt})


if not group_calculation:
    clear_arguments_file(cluster)


if name not in energy_transfer_results.keys():
    energy_transfer_results[name] = EnergyTransferResults(
        trials=trials_low_res,
        resolution=resolution,
        verbose=False,
        cluster=cluster,
        Ms=Ms,
        y_label=r"$\eta_{12}$",
        y_range=eta12_range,
        update_y=update_y,
        x_label=r"$\eta_2$",
        x_range=eta_range,
        update_x=update_x,
        dt=0.05,
        angle=0,
        rotation=True,
        print_MLE=False,
        title="D1={D1:.2f},\nD2={D2:.2f}, dt={dt}, L0={L0:.2f}, eta1={eta2:.2f}",
        figname_base=name,
    )
    energy_transfer_figures_list.append(name + ".pdf")

energy_transfer_results[name].run()

## Link strength and trajectory length $\lg B(\eta_{12}, M)$

In [ ]:
name = "localized_eta12_v_M_energy_transfer"


def update_y(args_dict, z):
    args_dict.update({"n12": z / dt})


def update_x(args_dict, z):
    args_dict.update({"M": z})


if not group_calculation:
    clear_arguments_file(cluster)

if name not in energy_transfer_results.keys():
    energy_transfer_results[name] = EnergyTransferResults(
        trials=trials_low_res,
        resolution=resolution,
        verbose=False,
        cluster=cluster,
        Ms=Ms,
        y_label=r"$\eta_{12}$",
        y_range=eta12_range,
        update_y=update_y,
        x_label=r"$M$",
        x_range=M_range,
        update_x=update_x,
        xscale="linear",
        dt=0.05,
        angle=0,
        rotation=True,
        print_MLE=False,
        title="D1={D1:.2f}, D2={D2:.2f},\nn1={n1:.2f}, n2={n2:.2f}, dt={dt}, L0={L0:.2f}",
        figname_base=name,
    )
    energy_transfer_figures_list.append(name + ".pdf")

energy_transfer_results[name].run(put_M_in_title=False)

## Diffusivity ratio and localization $\lg B(\gamma, \eta_1)$

In [ ]:
name = "localized_gamma_v_eta1_energy_transfer"


def update_y(args_dict, z):
    D1 = args_dict["D2"] * z
    args_dict.update({"D1": D1})


def update_x(args_dict, z):
    args_dict.update({"n1": z / dt})


if not group_calculation:
    clear_arguments_file(cluster)

if name not in energy_transfer_results.keys():
    energy_transfer_results[name] = EnergyTransferResults(
        trials=trials_high_res,
        resolution=resolution,
        verbose=False,
        cluster=cluster,
        Ms=Ms,
        y_label=r"$\gamma$",
        y_range=gamma_range,
        update_y=update_y,
        x_label=r"$\eta_1$",
        x_range=eta_range,
        update_x=update_x,
        dt=0.05,
        angle=0,
        rotation=True,
        print_MLE=False,
        title="D2={D2:.2f},\nn12={n12:.2f}, dt={dt}, L0={L0:.2f}, eta2={eta2:.2f}",
        figname_base=name,
    )
    energy_transfer_figures_list.append(name + ".pdf")

energy_transfer_results[name].run()

## Diffusivity ratio and localization $\lg B(\gamma, \eta_2)$

In [ ]:
name = "localized_gamma_v_eta2_energy_transfer"


def update_y(args_dict, z):
    D1 = args_dict["D2"] * z
    args_dict.update({"D1": D1})


def update_x(args_dict, z):
    args_dict.update({"n2": z / dt})


if not group_calculation:
    clear_arguments_file(cluster)

if name not in energy_transfer_results.keys():
    energy_transfer_results[name] = EnergyTransferResults(
        trials=trials_low_res,
        resolution=resolution,
        verbose=False,
        cluster=cluster,
        Ms=Ms,
        y_label=r"$\gamma$",
        y_range=gamma_range,
        update_y=update_y,
        x_label=r"$\eta_2$",
        x_range=eta_range,
        update_x=update_x,
        dt=0.05,
        angle=0,
        rotation=True,
        print_MLE=False,
        title="D2={D2:.2f},\nn12={n12:.2f}, dt={dt}, L0={L0:.2f}, eta1={eta1:.2f}",
        figname_base=name,
    )
    energy_transfer_figures_list.append(name + ".pdf")

energy_transfer_results[name].run()

## Diffusivity ratio and trajectory length $\lg B(\gamma, M)$

In [ ]:
name = "localized_gamma_v_M_energy_transfer"


def update_y(args_dict, z):
    D1 = args_dict["D2"] * z
    args_dict.update({"D1": D1})


def update_x(args_dict, z):
    args_dict.update({"M": z})


if not group_calculation:
    clear_arguments_file(cluster)

if name not in energy_transfer_results.keys():
    energy_transfer_results[name] = EnergyTransferResults(
        trials=trials_low_res,
        resolution=resolution,
        verbose=False,
        cluster=cluster,
        Ms=Ms,
        y_label=r"$\gamma$",
        y_range=gamma_range,
        update_y=update_y,
        x_label=r"$M$",
        x_range=M_range,
        update_x=update_x,
        xscale="linear",
        dt=0.05,
        angle=0,
        rotation=True,
        print_MLE=False,
        title="D2={D2:.2f},\nn1={n1:.2f}, n2={n2:.2f}, n12={n12:.2f}, dt={dt}, L0={L0:.2f}",
        figname_base=name,
    )
    energy_transfer_figures_list.append(name + ".pdf")

energy_transfer_results[name].run(put_M_in_title=False)